In [ ]:
# read in pdf and extract text
# We want to ignore all the decorum and only extract the text (i.e. no page number etc.)
import os
from pubscience.retrieve.OAI.parse_pdfs import pdf_to_text, extract_summary, text_extractor

In [ ]:
institute = 'Leiden'
pdf_path = f'//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PRETRAINING/PhDTheses/{institute}/theses/'

In [ ]:
Files = [f for f in os.listdir(pdf_path) if f.endswith('.pdf')]
Files.sort()

ListOfTexts = []    
ListOfSummaries = []
ListOfNumWordLists = []
ListOfErrors = []

In [ ]:
len(ListOfNumWordLists), len(Files)

In [ ]:
for k, _File in enumerate(Files):
    print(f"Processing file {k+1}/{len(Files)}: {_File}")
    _path = os.path.join(pdf_path, _File)
    try:
        Text, error, scanned, producer = pdf_to_text(_path)
    except Exception as e:
        print(f"Error processing {_File}: {e}")
        continue
    SummaryEnglish, SummaryDutch = extract_summary(Text)
    CleanedText, NumWordList = text_extractor(Text, min_words=30)

    for k, page in enumerate(CleanedText):
        ListOfTexts.append({
            'institute': institute,
            'file': _File,
            'pseudo_pagenum': k,
            'text': page
        })
    ListOfSummaries.append({
        'institute': institute,
        'file': _File,
        'summary_english': SummaryEnglish,
        'summary_dutch': SummaryDutch
    })
    ListOfNumWordLists.append({
        'institute': institute,
        'file': _File,
        'num_words_per_page': NumWordList
    })          
    ListOfErrors.append({
        'institute': institute,
        'file': _File,
        'error': error,
        'scanned': scanned,
        'pdf_producer': producer
    })
            

In [ ]:
sum([_r for r in ListOfNumWordLists for _r in r['num_words_per_page']])

In [ ]:
ListOfSummariesWith = [d for d in ListOfSummaries if (d['summary_english']!='') or (d['summary_dutch']!='')]
with_sum = len(ListOfSummariesWith)
ListOfSummariesWithout = [d for d in ListOfSummaries if (d['summary_english']=='') and (d['summary_dutch']=='')]
without_sum = len(ListOfSummariesWithout)
print(f'Out of {len(ListOfSummaries)} theses, {with_sum} have summaries and {without_sum} do not have summaries.')

In [ ]:
import json

with open(f'//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PRETRAINING/PhDTheses/{institute}_texts.jsonl', 'w', encoding='utf-8') as f:
    for item in ListOfTexts:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

In [ ]:
# the same for the summaries
with open(f'//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PRETRAINING/PhDTheses/{institute}_summaries.jsonl', 'w', encoding='utf-8') as f:
    for item in ListOfSummariesWith:
        f.write(json.dumps(item, ensure_ascii=True) + '\n')